#Pre-Question 1 Scripts

In [0]:
#code cell for extracting contents of zipf (data-lab3.zip) to google drive directory

#
#   SHOULD RUN THIS **ONLY** one time
#

import zipfile

zipf = 'data-lab3.zip'
gdrive = '/gdrive/My Drive/colab/'
file = gdrive+zipf
zi = zipfile.ZipFile(file, 'r')
zi.extractall(path='/gdrive/My Drive/colab')


FileNotFoundError: ignored

In [0]:
#code cell for mounting colab to google drive
#run this every time need to re-instantiate connection to colab servers

from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Question 1: Data

Approach:


1.   Get the path to the parent directory of the dog & cat folders (in this case,  /gdrive/My Drive/colab/data-lab3)
2.   Use imutils function paths.list_images to get a list of images, then sort the list
3.   Using a reproducable seed, shuffle the order of images
4.   For each image..

      A.  Read & resize/flatten image to 32x32x3
      
      B.  Append image to list X (images)
      
      C. Get label using the path seperator (parent folder name of image)
      
      D. Append label to list Y (labels)
      
      E.  If any error occurs during these steps, do not add image to X or label to Y.
5.   Print results of read
6.   conver X & Y to numpy arrays, then split the testing and training data
7.   Using pandas.factorize to categorize 'dog'  & 'cat' from string to integer (0 or 1, respectively)
8.   Using keras' to_categoricals to create a one-hot vector for this data.
9.   Create & display bar chart using the size of y_train and y_test



In [0]:
import cv2
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from imutils import paths
from sklearn.model_selection import train_test_split
from pandas import factorize
from keras.utils.np_utils import to_categorical

# base_path = '/gdrive/My Drive/colab/data-lab3'

# x=[]   #data
# y=[]   #labels

# imagePaths = sorted(list(paths.list_images(base_path)))
# random.seed(21)
# random.shuffle(imagePaths)
# n=0
# error = 0

# for imagePath in imagePaths:
#   n+=1
#   if (n % 1000 == 0):
#     print(str(n)+' images have been loaded')
#   try:
#     image = cv2.imread(imagePath)
#     image = cv2.resize(image, (28,28)).flatten()
#     x.append(image)
#     label = imagePath.split(os.path.sep)[-2]   #sets label as folder name (cat/dog)
#     y.append(label)
#   except:
#     print(imagePath + ' could not be read. (file size == 0)')
#     error+=1

    
# print('Finished reading files. %d successful, %d were not read.' % (n-error, error))
x = np.array(x, dtype='float') / 255.0   #normalizes pixel range from [0,255] to [0,1]
y = np.array(y)
#using to_categorical for one-hot encoding of categories
y = np.where(y_train == 'dog', 0, 1)
y_categorical =  to_categorical(y)

#maybe remove some dogs from numpy array (np.delete)

print(n)
print(len(x))

(x_train, x_test, y_train, y_test) = train_test_split(x,y_categorical,test_size=0.25, 
                                    train_size=0.75, random_state=21)


#display bar chart to show # of images in both training & testing categories 
heights = [len(sum(np.where(y_train == 'dog'))), len(sum(np.where(y_test == 'dog')))]
heights2 = [len(sum(np.where(y_train == 'cat'))), len(sum(np.where(y_test == 'cat')))]
xcoord = ['# training images', '# testing images']
plt.bar(x=xcoord, height=heights)
plt.bar(x=xcoord, height=heights2, bottom = heights)
plt.annotate('dog size='+str(heights[0]), xy=(-0.18,heights[0]/2))
plt.annotate('dog size='+str(heights[1]), xy=(0.8,heights[1]/2))
plt.annotate('cat size='+str(heights2[0]), xy=(-0.18,heights2[0]*2.8))
plt.annotate('cat size='+str(heights2[1]), xy=(0.82,heights2[1]*2.5))

plt.show()

7390
7384


ValueError: ignored

# Question 2: Classification

Approach:


1.   Create sequential NN
2.   Create input layer (shape 3072, since  32x32x3 = 3072 )
3.   Create 3 hidden layers, each with progressively fewer neurons. With Dropouts between them to help regularize predictions.
4.   Output layer has an output size of 2 because we have 2 categories
5.   create SGD optimizer using keras
6.   compile the model using recommended parameters (binary_crossentropy b/c we are only resulting in 2 categories)
7.   Fit the model using training & testing data, for 30 epochs.
8.   Print summary of data.


In [0]:
from keras.layers import Dense, Dropout
import keras


#create neural network
model = keras.Sequential()
#input shape of 2352 because images are flattened to 28x28x3 = 2352 pixels
model.add(Dense(1024, input_shape=(2352,), activation='sigmoid'))
model.add(Dropout(0.25))
#3 hidden layers
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.25))
#output shape of 2 because we have 2 categories, cat or dog.
model.add(Dense(len(np.unique(y_train_label)), activation='softmax'))

EPOCHS = 60
#small learning rate is better for computer vision
INIT_LR = 0.001

#compile model using compile from keras.
#can use a for loop for different SGD values to see which works best
sgd = keras.optimizers.SGD(lr=INIT_LR)
#also try Adam
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


#use fit from keras to train model. Pass in training data and 
#testing data seperately, & choose values for inputs (like epochs)

#class weights since there are more dog images than cat images
class_weight = {0: 1, 1: 2.05}

M = model.fit(x_train, y_train, class_weight=class_weight, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=32)

model.summary()

Train on 5538 samples, validate on 1846 samples
Epoch 1/60
5538/5538 [==============================] - 8s 1ms/step - loss: 0.9815 - acc: 0.5390 - val_loss: 0.6879 - val_acc: 0.6641
Epoch 2/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9733 - acc: 0.5052 - val_loss: 0.6774 - val_acc: 0.6641
Epoch 3/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9747 - acc: 0.5159 - val_loss: 0.6851 - val_acc: 0.6641
Epoch 4/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9828 - acc: 0.5007 - val_loss: 0.6804 - val_acc: 0.6641
Epoch 5/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9767 - acc: 0.5079 - val_loss: 0.6889 - val_acc: 0.6641
Epoch 6/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9732 - acc: 0.5058 - val_loss: 0.6948 - val_acc: 0.3359
Epoch 7/60
5538/5538 [==============================] - 7s 1ms/step - loss: 0.9657 - acc: 0.5103 - val_loss: 0.6947 - val_acc: 0.3359
Epoch 8/60
553

# Question 3: Evaluation

 1. get a predictions vector by passing in our testing data
 2. compare prediction data against our testing data using scikit-learn's classification report
 3. get false positive & false negative rates using scikit-learn's roc_curve method. (using our testing data labels as 'y_true')
 4. Visualize ROC_CURVE results using matplotlib   #NOTE: my ROC curve is TERRIBLE!! haha
 
 NOTE: my model predicts dog 100% of the time since it has a higher confidence in the dog label for every datapoint.
 I have little idea why it isn't learning from its previous mistakes. Most likely related to number of samples for each category. I tried to offset this using class_weights during model.fit, but it didn't help as much as expected.


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

#test model by feeding testing dataset (images only)
predictions = model.predict(x_test)

#compare returned predictions to the known labels using classification_report

print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=['dog', 'cat']))

y_true = y_test.argmax(axis=1)
y_score = predictions[:,0]
#fpr: Increasing false positive rates such that element i is the false positive rate of predictions with score >= thresholds[i].
#tpr: Increasing true positive rates such that element i is the true positive rate of predictions with score >= thresholds[i].
#threshes: Decreasing thresholds on the decision function used to compute fpr and tpr. thresholds[0] represents no instances being predicted and is arbitrarily set to max(y_score) + 1.
fpr, tpr, threshes = roc_curve(y_true, y_score)


#viz results by displaying ROC curve - a probability curve plotting the true 
  #positive rate against the false positive rate.
  #put FPR on x-axis and TPR on y-axis.
  #use roc_curve from scikit-learn
  
  
  
plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

NameError: ignored